In [169]:
import numpy as np
import random
from random import randint
import gmpy2

Допоміжні ф-ї

In [170]:
def conv_to_list(n, t):
    num = [int(i) for i in bin(n)[2:]]
    while len(num) < t:
        num.insert(0, 0)
    return num

def display(lst):
    for i in lst:
        print(i)
        
def bin_lst_to_int(lst):
    return int("".join(str(x) for x in lst), 2)

def gen_bits(N):
    d = np.random.randint(0, 2, N).tolist()
    d_int = bin_lst_to_int(d)
    if d_int == 0: #check if all elements in d are 0
        while d_int == 0:
            d = np.random.randint(0, 2, N).tolist()
            d_int = bin_lst_to_int(d)
            print(d)
    return d

def hex_to_list(hex, b):
    n = int(hex, 16)
    number = []
    while n > 0:
        number.insert(0, n % b)  # take remainder of division and add it leftmost of the array
        n = n // b
    return number

def hex_to_int(hex):
    return bin_lst_to_int(hex_to_list(hex, 2))

1. Вбудований генератор псевдовипадкових чисел

In [171]:
def rand(N):
    random.seed(1)
    for i in range(0,N):
        num = randint(0, 256)
        bin_num = bin(num)[2:]
        n = []
        for i in range(0,len(bin_num)): n.append(int(bin_num[i]))
        while len(n) != 8: n.insert(0,0)
        print(n)

rand (10)

[0, 1, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 1, 1, 1, 1, 0, 0]
[1, 1, 1, 1, 1, 1, 0, 1]
[1, 1, 1, 0, 0, 1, 1, 0]
[1, 1, 1, 1, 0, 0, 0, 1]
[1, 1, 0, 0, 0, 0, 1, 0]
[0, 1, 1, 0, 1, 0, 1, 1]
[0, 0, 1, 1, 0, 0, 0, 0]


2. Генератор LehmerLow

In [172]:
m = 2**32
a = 2**16 + 1
c = 119

def LehmerLow(N):
    prev = np.random.randint(0,2**31) 
    res = []
    for i in range(N):
        x_i = (a * prev + c)%m
        res_i = conv_to_list(x_i, 32)[-8:]
        res.append(res_i)
        prev = x_i
    return res    

lehmer_low = LehmerLow(10)
display(lehmer_low)

[1, 0, 1, 0, 0, 1, 1, 0]
[0, 0, 0, 1, 1, 1, 0, 1]
[1, 0, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 0, 0, 0, 1, 0]
[1, 1, 1, 1, 1, 0, 0, 1]
[0, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 1, 1, 1]
[0, 1, 0, 1, 1, 1, 1, 0]
[1, 1, 0, 1, 0, 1, 0, 1]


3. Генератор LehmerHigh

In [173]:
m = 2**32
a = 2**16 + 1
c = 119

def LehmerHigh(N):
    prev = np.random.randint(0,2**31) 
    res = []
    for i in range(N):
        x_i = (a * prev + c)%m
        res_i = conv_to_list(x_i, 32)[:8]
        res.append(res_i)
        prev = x_i
    return res    

lehmer_high = LehmerHigh(10)
display(lehmer_high)

[1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 1, 0, 1]
[0, 0, 1, 0, 1, 0, 1, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 1, 1, 0]
[0, 0, 1, 0, 1, 1, 0, 1]
[1, 0, 0, 0, 0, 1, 0, 0]
[1, 1, 0, 1, 1, 0, 1, 1]
[0, 0, 1, 1, 0, 0, 1, 1]
[1, 0, 0, 0, 1, 1, 0, 0]


4. Генератор псевдовипадкових двійкових послідовностей L20

In [174]:
def L20(N):
    d1 = np.random.randint(0, 2, 20)
    d = d1.tolist()
    print(d)
    for i in range(20, 20 + N): 
        x = d[i-3] ^ d [i-5] ^ d[i-9] ^ d[i-20]
        d.append(x)
        
    return d

l20 = L20(16)
print(l20, len(l20))

[1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0]
[1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1] 36


5. Генератор псевдовипадкових двійкових послідовностей L89

In [175]:
def L89(N):
    d = gen_bits(89)
    print(d)
    for i in range(89, 89 + N): 
        x = d[i-38] ^ d [i-89]
        d.append(x)
    return d

l89 = L89(16)
print(l89, len(l89))

[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1] 105


6. Генератор Джиффі

In [176]:
def L11(n):
    d1 = np.random.randint(0, 2, 10)
    d = d1.tolist()
    for i in range(11, n+1): 
        x = d[i-11] ^ d [i-9]
        d.append(x)
    return d

def L9(n):
    d1 = np.random.randint(0, 2, 8)
    d = d1.tolist()
    for i in range(9, n+1): 
        x = d[i-9] ^ d [i-8] ^ d[i-6] ^ d[i-5]
        d.append(x)
    return d

def L10(n):
    d1 = np.random.randint(0, 2, 9)
    d = d1.tolist()
    for i in range(10, n+1): 
        #print(len(d))
        x = d[i-10] ^ d [i-7]
        d.append(x)
    return d

def Geffe(n):
    d = []
    l11 = L11(n)
    l9 = L9(n)
    l10 = L10(n)
    #print(len(l11),len(l9),len(l10))
    for i in range(0,n):
        x = l11[i]
        y = l9[i]
        s = l10[i]
        #z = (s * x) ^ ((1 ^ s) * y)
        if s == 1: z = x
        elif s == 0: z = y
        else: print('error, s = ', s)
        d.append(z)
    return d

print(Geffe(18), len(Geffe(18)))

[0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0] 18


7. Генератор Вольфрама

In [177]:
def Wolfram(N):
    r = gen_bits(32)
    res = []
    for i in range(N):
        x_i = r[-1]
        r = np.roll(r, -1) ^ (r | np.roll(r, 1)) 
        res.append(x_i)
    return res

wolfram = Wolfram(20)
print(wolfram)

[0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1]


8. Генератор "Бібліотекар"

In [188]:
def librarian():
    f = open(r'achilles.txt', 'r', encoding='utf8', errors='ignore')
    text = f.read()
    f.close()
    b = text.encode('utf-8')
    gen_nums = []
    for i in range(0,len(b)): gen_nums.append(conv_to_list(b[i],8))
    return gen_nums

display(librarian()[:50])

[1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 1, 0, 1]
[1, 1, 0, 1, 0, 0, 0, 1]
[1, 0, 0, 0, 1, 0, 0, 1]
[1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 1, 0, 1]
[0, 0, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 1, 1, 0]
[1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 1, 0, 0]
[1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 1, 0, 1]
[1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 1, 1, 0]
[1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 1, 1]
[1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 1, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 1, 1]
[1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 1, 0, 1]
[1, 1, 0, 1, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 1, 1, 0]
[1, 1, 0, 1, 0, 0, 0, 1]
[1, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 1, 0, 0]
[1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 1, 1, 0]
[1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 0, 1, 1]
[1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 1, 1, 0]
[1, 1, 0, 1, 0, 0, 0, 0]


9. Генератор Блюма-Мікалі (BM)

In [179]:
P = "CEA42B987C44FA642D80AD9F51F10457690DEF10C83D0BC1BCEE12FC3B6093E3"
A = "5B88C41246790891C095E2878880342E88C79974303BD0400B090FE38A688356"
Q = "675215CC3E227D3216C056CFA8F8822BB486F788641E85E0DE77097E1DB049F1"

def BM(N):
    p = hex_to_int(P)
    a = hex_to_int(A)
    q = hex_to_int(Q)
    T = random.randint(0,p-1)
    res = []
    for i in range(N):
        if T < gmpy2.c_div(p-1, 2):
            x = 1
        else:
            x = 0
        res.append(x)
        T = gmpy2.powmod(a,T,p)
    return res

blum_micali = BM(20)
print(blum_micali)

[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1]


10. Генератор BM_bytes (байтовий)

In [185]:
P = "CEA42B987C44FA642D80AD9F51F10457690DEF10C83D0BC1BCEE12FC3B6093E3"
A = "5B88C41246790891C095E2878880342E88C79974303BD0400B090FE38A688356"
Q = "675215CC3E227D3216C056CFA8F8822BB486F788641E85E0DE77097E1DB049F1"

def BM_bytes(N):
    p = hex_to_int(P)
    a = hex_to_int(A)
    q = hex_to_int(Q)
    T = random.randint(0,p-1)
    res = []
    for i in range(N):
        g = gmpy2.c_div(gmpy2.mul(256,T), p-1)
        for k in range(g-1, g):
            if k < g and k >= g-1:
                x = conv_to_list(k, 8)
                res.append(x)
                break
        T = gmpy2.powmod(a,T,p)
    return res

bm_bytes = BM_bytes(10)
display(bm_bytes)

[1, 1, 0, 0, 1, 0, 1, 1]
[1, 0, 1, 1, 0, 1, 1, 1]
[1, 0, 1, 0, 1, 0, 0, 0]
[1, 1, 0, 1, 1, 1, 1, 0]
[1, 0, 0, 1, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 1]
[1, 1, 0, 1, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 1, 0, 0]
[0, 1, 0, 1, 0, 1, 0, 1]
[1, 1, 0, 0, 1, 1, 0, 0]


11. Генератор BBS (бітовий)

In [181]:
p = 'D5BBB96D30086EC484EBA3D7F9CAEB07'
q = '425D2B9BFDB25B9CF6C416CC6E37B59C1F'
n = int(p,16) * int(q,16)

def BBS_bin(N):
    
    r = [randint(0,n)]
    x = []
    for i in range (1,N):
        R = (r[i-1] ** 2) % n
        r.append(R)
        x.append(R % 2)
    return x
    
    
x = BBS_bin(20)
print(x)

[0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1]


12. Генератор BBS (байтовий)

In [184]:
def BBS_b(N):
    
    r = [randint(0,n)]
    x = []
    for i in range (1,N):
        R = (r[i-1] ** 2) % n
        r.append(R)
        x.append(conv_to_list(R % 256, 8))
    return x
    
x = BBS_b(20)
display(x)

[1, 0, 1, 1, 0, 0, 1, 1]
[0, 1, 1, 0, 1, 1, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 1, 1]
[1, 0, 0, 0, 0, 1, 0, 1]
[1, 0, 1, 1, 0, 1, 1, 1]
[0, 0, 0, 1, 1, 0, 0, 1]
[1, 0, 0, 0, 1, 1, 0, 1]
[1, 1, 0, 1, 0, 0, 1, 1]
[0, 0, 1, 0, 1, 1, 0, 0]
[1, 0, 0, 1, 0, 1, 1, 1]
[1, 1, 0, 1, 1, 0, 1, 1]
[1, 1, 0, 0, 1, 1, 0, 1]
[1, 0, 0, 0, 1, 1, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 1]
[0, 1, 0, 1, 0, 0, 1, 0]
[0, 1, 1, 1, 0, 1, 0, 1]
[0, 1, 1, 1, 0, 0, 0, 1]
[1, 0, 0, 0, 1, 0, 0, 0]
